In [1]:
import sys; sys.path.append("../")

import pandas as pd

from clean.post_pprr import clean
from lib.uid import gen_uid
from lib.date import combine_date_columns
from lib.match import (
    ThresholdClassifier, ColumnsIndex, StringSimilarity, JaroWinklerSimilarity,
    DateSimilarity, match_records, print_match_result
)

In [2]:
post = clean()
post

,agency,last_name,first_name,employment_status,level_1_cert_date,last_pc_12_qualification_date,hire_year,hire_month,hire_day,uid
0,14th jdc da office,abshire,dustin,full-time,06/11/1998,10/27/2010,2013,06,01,43ee7cb8a15ca49e883372d3c99988e1
1,17th da office,ourso,damian,full-time,05/13/2004,10/07/2010,2008,11,11,8875f7c0749a1de0923c1242a4363622
2,17th da office,besson,bruce,full-time,NaN,01/26/2008,1997,02,11,2c35336b17f9986c7547656bb72508fe
3,17th da office,bourg,billy,full-time,04/24/1991,10/07/2010,1990,10,12,e17ebe5f0a893d9cb6aac11bf9673c32
4,19th da office,cadarette,steve,full-time,08/28/2003,04/28/2020,2016,04,11,84a22650eb29e5d7d17721d59c3a2fc6
...,...,...,...,...,...,...,...,...,...,...
38898,youngsville pd,fruge,hilary,reserve,11/18/2010,06/22/2020,2017,11,27,2e785e632588b75ebe30c3651f80837c
38899,youngsville pd,mouton,todd,reserve,NaN,06/15/2020,2017,07,01,e2642ae3d8360270ac3b4cfd5d8c5ea4
38900,youngsville pd,hidalgo,jeremy,reserve,NaN,07/31/2020,2017,12,01,a5ca9eebadff5a86ea4e7dd507ba1f73
38901,zachary pd,viccellio,david,reserve,NaN,12/04/2019,2019,12,03,3c6043ea0d02ebbab16968c88c9107f0


In [3]:
from clean.new_orleans_csd_pprr import clean_2014
no_pprr_14 = clean_2014()
no_pprr_14

,department_desc,last_name,first_name,rank_code,rank_desc,annual_salary,birth_year,birth_month,birth_day,hire_year,...,hire_day,term_year,term_month,term_day,pay_prog_start_year,pay_prog_start_month,pay_prog_start_day,mid,data_production_year,agency
0,pol managemnt services bureau,moore,charmaine,C0025,clerical specialist,11235.83,1901,1,1,1996,...,8,1996,8,28,1996,8,28,d9b2bb8829f4b2e4fd97290ea93ee56e,2014,New Orleans CSD
1,pol managemnt services bureau,coxen,shanna,C5201,laboratory technician 1,14951.72,1972,10,5,1996,...,22,1997,1,1,1997,1,1,693a1967b92e800cfca8b5f800030d47,2014,New Orleans CSD
2,pol special investigation div,cavaliere sr,morris,C7134,police officer 4,29243.06,1947,7,21,1970,...,21,1997,1,4,1997,1,4,0e1583f54d720cd899d9ce17c5d12869,2014,New Orleans CSD
3,pol first district,fonseca,norbert,C7112,police lieutenant,39893.17,1943,8,31,1967,...,21,1997,1,4,1997,1,4,fff2935b97640caa5e15508cf21a871c,2014,New Orleans CSD
4,pol fifth district,hartman,steven,C7112,police lieutenant,33524.95,1964,3,11,1985,...,10,1997,1,4,1997,1,4,7b5f6f0e0cd3b2cfe3718d64a535ac43,2014,New Orleans CSD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3681,pol managemnt services bureau,johnson,shawn,C7105,police aide,28882.03,1990,6,23,2014,...,17,2014,12,10,2014,8,17,3e28e9f4250a2e3ae76d6f36345c6153,2014,New Orleans CSD
3682,pol sixth district,linehan,henry,C7110,police officer 1,37490.69,1982,8,18,2009,...,27,2014,12,11,2011,3,13,7693cd75ee286e1b018fe5df832f79dc,2014,New Orleans CSD
3683,pol first district,finneman,david,C7111,police sergeant,50513.12,1960,12,20,2001,...,16,2014,12,13,2013,1,30,a4180ae4b9be2d1a19f2f68088cc6254,2014,New Orleans CSD
3684,pol managemnt services bureau,badr,mohamed,C7105,police aide,28882.03,1980,10,6,2014,...,31,2014,12,20,2014,8,31,eb56dc40d6c443298c510e1925494a56,2014,New Orleans CSD


In [9]:
no_post = post.loc[post.agency == 'new orleans pd', ['uid', 'first_name', 'last_name']]
no_post.loc[:, "hire_date"] = combine_date_columns(post, "hire_year", "hire_month", "hire_day")
no_post = no_post.drop_duplicates(["uid"]).set_index("uid", drop=True)

no_personnel = no_pprr_14.loc[:, ['mid', 'first_name', 'last_name']]
no_personnel.loc[:, "hire_date"] = combine_date_columns(no_pprr_14, "hire_year", "hire_month", "hire_day")
no_personnel = no_personnel.drop_duplicates(["mid"]).set_index("mid", drop=True)

matches, potential_matches, non_matches = match_records(
    ColumnsIndex(["first_name"]),
    ThresholdClassifier(
        fields={
            "last_name": JaroWinklerSimilarity(),
            "hire_date": DateSimilarity()
        },
        thresholds=[0.8, 0.5]
    ),
    no_post,
    no_personnel
)


In [20]:
print("POST match percentage: %d%%" % (len(matches) / no_post.shape[0] * 100))
print("NO CSD PPRR match percentage: %d%%" % (len(matches) / no_personnel.shape[0] * 100))
print_match_result(no_post, no_personnel, matches, potential_matches, [])

POST match percentage: 7%
NO CSD PPRR match percentage: 3%
matches: 114
potential matches: 453
non-matches: 0

*** MATCHES ***

                                 first_name last_name  hire_date
6acb2f47bd3af11dcd64fc747d34f752       tony   jackson 2004-10-17
3890ae2b6a2181a0afff4f09e286e168       tony   jackson 2004-10-17

                                 first_name  last_name  hire_date
3b322009968f44dd5960513db876d443      keith   mcelrath 1976-12-13
2e3a695b1bab7cf04a254561e281bc30      keith  mc elrath 1976-12-20

                                 first_name last_name  hire_date
c76ec8aed8eb4c6c0139860a5dd6f841      keith    jerome 1998-05-10
b632ea5789eb88290383638ff030abf1      keith    jerome 1998-05-10

                                 first_name last_name  hire_date
016639c27f05cfda3e0582fb71e4e7e6      lamar     james 1999-12-26
4f458677d57f028cbaf6e6eb0b0b35e0      lamar     james 1999-12-26

                                 first_name last_name  hire_date
99e565a9b676ed94ef7b